# Lab 7B: Foundry Tool Catalog

When creating agents, you can attach tools directly in the agent definition. This **ties the tool configuration to the agent** - every agent that needs the tool must include the full configuration:

```python
# Direct approach: Tool config embedded in agent definition
agent = project_client.agents.create_version(
    agent_name="my-agent",
    definition=PromptAgentDefinition(
        model=MODEL,
        instructions="...",
        tools=[{
            "type": "mcp",
            "server_label": "mslearn",
            "server_url": "https://learn.microsoft.com/api/mcp",  # Hardcoded URL
            "require_approval": "never"
        }]
    )
)
```

The **Foundry Tool Catalog** solves this by providing a centralized registry. Register your MCP servers and OpenAPI credentials once, then reference them by connection ID:

```python
# Catalog approach: Reference tool by connection ID
mcp_tool = project_client.connections.get("mslearn-mcp")

agent = project_client.agents.create_version(
    agent_name="my-agent",
    definition=PromptAgentDefinition(
        model=MODEL,
        instructions="...",
        tools=[{
            "type": "mcp",
            "server_label": "mslearn",
            "server_url": mcp_tool.target,           # From catalog
            "project_connection_id": mcp_tool.id,    # Reference to catalog
            "require_approval": "never"
        }]
    )
)
```

## Why Use the Tool Catalog?

| Direct Embedding | Tool Catalog |
|------------------|--------------|
| Tool config hardcoded in each agent | Register once, reference by ID |
| Rotating credentials = update every agent | Rotate credentials in one place |
| No visibility into which tools exist | Centralized registry of approved tools |
| Teams duplicate tool configurations | Teams share approved tools |
| Credentials scattered across agent definitions | Credentials stored securely in connections |

## Lab Flow

1. **Register Tools** - Add MCP server and OpenAPI credentials to the catalog
2. **List Tools** - View all registered tools
3. **Use in Agent** - Create an agent that uses a cataloged tool (no credential config needed)
4. **Reuse Across Agents** - Multiple agents share the same tool

---

## Setup

In [ ]:
import os
import json
import subprocess
import requests
from azure.identity import DefaultAzureCredential
from IPython.display import display, HTML, Markdown

# Load environment
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

SPOKE_ACCOUNT = os.environ['SPOKE_ACCOUNT']
SPOKE_PROJECT = os.environ['SPOKE_PROJECT']
PROJECT_ENDPOINT = f"https://{SPOKE_ACCOUNT}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"
APIM_CONNECTION = os.environ['APIM_CONNECTION']
MODEL_NAME = os.environ['MODEL_NAME']
GATEWAY_MODEL = f"{APIM_CONNECTION}/{MODEL_NAME}"

# Get subscription
result = subprocess.run(["az", "account", "show", "-o", "json"], capture_output=True, text=True)
SUBSCRIPTION_ID = json.loads(result.stdout)["id"]
RESOURCE_GROUP = "foundry-child-1"

# ARM API setup
credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default")
ARM_BASE = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{SPOKE_ACCOUNT}/projects/{SPOKE_PROJECT}"
API_VERSION = "2025-04-01-preview"
headers = {"Authorization": f"Bearer {token.token}", "Content-Type": "application/json"}

print(f"Project: {SPOKE_PROJECT}")
print(f"Account: {SPOKE_ACCOUNT}")
print(f"Model: {GATEWAY_MODEL}")

---

## Part 1: Register Tools in the Catalog

Add tools to the catalog using the ARM REST API. The catalog stores:
- **MCP Servers** - Endpoint URL for Model Context Protocol servers
- **OpenAPI Credentials** - API keys for external services

In [2]:
def register_mcp_tool(name: str, endpoint_url: str):
    """Register an MCP server in the Tool Catalog."""
    url = f"{ARM_BASE}/connections/{name}?api-version={API_VERSION}"
    payload = {
        "properties": {
            "category": "RemoteTool",
            "authType": "None",
            "target": endpoint_url,
            "metadata": {"type": "custom_MCP"}
        }
    }
    response = requests.put(url, headers=headers, json=payload)
    return response.status_code in [200, 201]

def register_openapi_tool(name: str, api_keys: dict):
    """Register OpenAPI credentials in the Tool Catalog."""
    url = f"{ARM_BASE}/connections/{name}?api-version={API_VERSION}"
    payload = {
        "properties": {
            "category": "CustomKeys",
            "authType": "CustomKeys",
            "target": "-",
            "isSharedToAll": True,
            "credentials": {"keys": api_keys},
            "metadata": {"type": "openapi"}
        }
    }
    response = requests.put(url, headers=headers, json=payload)
    return response.status_code in [200, 201]

print("Helper functions ready")

Helper functions ready


In [3]:
# Register an MCP Server
MCP_TOOL_NAME = "mslearn-mcp"
MCP_ENDPOINT = "https://learn.microsoft.com/api/mcp"

if register_mcp_tool(MCP_TOOL_NAME, MCP_ENDPOINT):
    print(f"Registered MCP tool: {MCP_TOOL_NAME}")
    print(f"  Endpoint: {MCP_ENDPOINT}")
else:
    print(f"Failed to register {MCP_TOOL_NAME}")

Registered MCP tool: mslearn-mcp
  Endpoint: https://learn.microsoft.com/api/mcp


In [4]:
# Register OpenAPI credentials for Petstore API (classic demo service)
# https://petstore.swagger.io - no auth required, but this demonstrates credential storage
OPENAPI_TOOL_NAME = "petstore-sample-api"

if register_openapi_tool(OPENAPI_TOOL_NAME, {"api_key": "demo-key"}):
    print(f"Registered OpenAPI tool: {OPENAPI_TOOL_NAME}")
    print(f"  API key stored securely in catalog")
    print(f"  Endpoint: https://petstore.swagger.io/v2")
else:
    print(f"Failed to register {OPENAPI_TOOL_NAME}")

Registered OpenAPI tool: petstore-sample-api
  API key stored securely in catalog
  Endpoint: https://petstore.swagger.io/v2


---

## Part 2: List All Tools in the Catalog

View all registered tools. Tools are identified by their category and metadata type.

In [5]:
def list_tools():
    """List all tools in the Foundry Tool Catalog."""
    url = f"{ARM_BASE}/connections?api-version={API_VERSION}"
    response = requests.get(url, headers=headers)
    connections = response.json().get("value", [])
    
    mcp_tools = []
    openapi_tools = []
    
    for conn in connections:
        props = conn.get("properties", {})
        category = props.get("category", "")
        metadata_type = props.get("metadata", {}).get("type", "")
        
        if category == "RemoteTool" and metadata_type == "custom_MCP":
            mcp_tools.append({"name": conn["name"], "target": props.get("target", "")})
        elif category == "CustomKeys" and metadata_type == "openapi":
            openapi_tools.append({"name": conn["name"]})
    
    return mcp_tools, openapi_tools

# List tools
mcp_tools, openapi_tools = list_tools()

display(Markdown("## Tool Catalog"))

print(f"MCP Servers ({len(mcp_tools)}):")
for tool in mcp_tools:
    print(f"  - {tool['name']}: {tool['target']}")

print(f"\nOpenAPI Credentials ({len(openapi_tools)}):")
for tool in openapi_tools:
    print(f"  - {tool['name']}")

## Tool Catalog

MCP Servers (1):
  - mslearn-mcp: https://learn.microsoft.com/api/mcp

OpenAPI Credentials (1):
  - petstore-sample-api


In [ ]:
# Generate a link to view tools in the Foundry Portal
import uuid
import base64

sub_bytes = uuid.UUID(SUBSCRIPTION_ID).bytes
encoded_sub = base64.urlsafe_b64encode(sub_bytes).decode('utf-8').rstrip('=')

TOOLS_PORTAL_URL = (
    f"https://ai.azure.com/nextgen/r/{encoded_sub},{RESOURCE_GROUP},,{SPOKE_ACCOUNT},{SPOKE_PROJECT}"
    f"/Build/tools"
)

display(Markdown(f'''
### View Tools in Foundry Portal

You can also browse and manage your registered tools directly in the Azure AI Foundry Portal:

**[Open Tool Catalog in Portal]({TOOLS_PORTAL_URL})**

The portal provides a visual interface to:
- View all registered MCP servers and OpenAPI connections
- Edit tool configurations
- Delete unused tools
- Test tool connectivity
'''))

---

## Part 3: Use a Tool in an Agent

Create an agent that uses a cataloged tool. The key benefit: **no need to configure credentials** - they're already stored in the catalog connection.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition

# Create project client
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# Get the MCP tool from the catalog
mcp_tool = project_client.connections.get(MCP_TOOL_NAME)

print(f"Retrieved tool from catalog:")
print(f"  Name: {mcp_tool.name}")
print(f"  Endpoint: {mcp_tool.target}")
print(f"  Connection ID: {mcp_tool.id}")

In [8]:
# Create an agent using the cataloged MCP tool
AGENT_NAME = "docs-assistant"

agent = project_client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="You help users find information from Microsoft Learn documentation using the MCP tools.",
        tools=[
            {
                "type": "mcp",
                "server_label": "mslearn",
                "server_url": mcp_tool.target,           # From catalog
                "project_connection_id": mcp_tool.id,    # Reference to catalog entry
                "require_approval": "never"
            }
        ]
    ),
)

print(f"Agent created: {agent.name} v{agent.version}")

Agent created: docs-assistant v1


In [9]:
# Run the agent
openai_client = project_client.get_openai_client()

response = openai_client.responses.create(
    input="What is Azure Functions and how do I create one?",
    extra_body={
        "agent": {
            "name": agent.name,
            "version": agent.version,
            "type": "agent_reference"
        }
    }
)

print(f"User: What is Azure Functions and how do I create one?")
print(f"\nAgent: {response.output_text}")

User: What is Azure Functions and how do I create one?

Agent: Azure Functions is a serverless compute service that enables you to run event-driven code blocks called "functions." It allows you to implement your system’s logic in an easily scalable manner without worrying about infrastructure. Azure Functions supports multiple programming languages and various triggers to respond to different events.

To create an Azure Function, you can use several tools including:
1. Azure Developer CLI (azd)
2. Command line interface (Azure CLI)
3. Visual Studio Code
4. Visual Studio

You can create and deploy your first functions in less than five minutes using any of these tools.

Detailed information and quickstart guides for creating Azure Functions in various programming languages (C#, JavaScript, Python, PowerShell, TypeScript) are available. Here are some useful links for getting started:

- Overview of Azure Functions and getting started guides:  
  https://learn.microsoft.com/en-us/azure/az

---

## Summary

### Tool Catalog Connection Types

| Tool Type | Category | Auth Type | Metadata Type |
|-----------|----------|-----------|---------------|
| MCP Server | `RemoteTool` | `None` | `custom_MCP` |
| OpenAPI Keys | `CustomKeys` | `CustomKeys` | `openapi` |

### Key Benefits

1. **Register Once** - Add tools to catalog via ARM API
2. **Credentials Stored Securely** - API keys in the connection, not in agent definitions
3. **Reuse Everywhere** - Multiple agents reference the same tool by connection ID  
4. **Easy Rotation** - Update credentials in one place, all agents get the update

---

## Cleanup

In [ ]:
# Delete agents
project_client.agents.delete_version(agent_name=AGENT_NAME, agent_version=agent.version)
print("Agents deleted")

# Optionally delete tool connections (uncomment to run)
# requests.delete(f"{ARM_BASE}/connections/{MCP_TOOL_NAME}?api-version={API_VERSION}", headers=headers)
# requests.delete(f"{ARM_BASE}/connections/{OPENAPI_TOOL_NAME}?api-version={API_VERSION}", headers=headers)
# print("Tool connections deleted")